In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('drugs_data.csv', delimiter=";")

In [8]:
df = df[['drug_name', 'uses']]
df.to_csv('cleaned.csv', index=False)

In [9]:
cleaned = pd.read_csv("cleaned.csv")
print(cleaned.info())
# Remove rows where 'drug_name' contains a comma


# Remove rows where 'drug_name' is empty or contains only commas
cleaned = cleaned[cleaned['drug_name'].notna()]  # Remove NaN values
cleaned = cleaned[~cleaned['drug_name'].str.contains(r'^[, ]*$', na=False)]  # Remove empty/comma-only rows
cleaned = cleaned[~cleaned['drug_name'].str.contains(',', na=False)]  # Remove rows where 'drug_name' contains commas

# Keep only 'drug_name' and 'summary' columns
final_df = cleaned[['drug_name', 'uses']]

# Save the cleaned dataset
final_df.to_csv('final.csv', index=False)

print("Final file saved without drug names containing commas or empty values.")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1424 entries, 0 to 1423
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   drug_name  571 non-null    object
 1   uses       571 non-null    object
dtypes: object(2)
memory usage: 22.4+ KB
None
Final file saved without drug names containing commas or empty values.


In [10]:
print(final_df.head())

           drug_name                                               uses
0             Abecma  Abecma is a CAR T cell therapy used to treat r...
1            Abilify  Abilify is an antipsychotic medication. It wor...
2  Abilify Asimtufii  Abilify Asimtufii is an atypical antipsychotic...
3   Abilify Maintena  Abilify Maintena (aripiprazole) extended-relea...
4        Abiraterone  Abiraterone works by reducing androgen product...


In [11]:
print(final_df.isnull().sum())

drug_name    0
uses         0
dtype: int64


In [12]:
final_df["text_length"] = final_df["uses"].dropna().apply(len)
print(final_df["text_length"].describe())

count      535.000000
mean       673.590654
std        881.275116
min          9.000000
25%        276.000000
50%        450.000000
75%        760.000000
max      12756.000000
Name: text_length, dtype: float64


In [13]:
from collections import Counter
import re

Average, Max & Min length of 'uses' Description


In [14]:
final_df["text_length"] = final_df["uses"].dropna().apply(len)
print("\nAverage text length (characters not words):", final_df['text_length'].mean()) 
print("Max text legnth (characters not words):", final_df['text_length'].max())
print("Min text legnth (characters not words):", final_df['text_length'].min())


Average text length (characters not words): 673.5906542056075
Max text legnth (characters not words): 12756
Min text legnth (characters not words): 9


Drugs with the Longest 'uses' Descriptions

In [15]:
print("\nDrugs with the longest Description:") 
print(final_df.loc[final_df['text_length'].nlargest(10).index, ['drug_name', 'text_length']])


Drugs with the longest Description:
         drug_name  text_length
567        Alcohol        12756
1201  Atezolizumab         6701
10      Adalimumab         4215
311     Adalimumab         4215
1029       Arcoxia         3927
716       Amjevita         3630
111       Abrilada         3578
62         Austedo         3290
1283       Austedo         3290
8          Actemra         2969


In [20]:
# Extract the first 10 rows, bch l'operation te5ou a9al wa9t
First10 = final_df.head(10)

# Save it to a new CSV file
First10.to_csv('First10.csv', index=False)

print("First10 file created with the first 10 rows!")


First10 file created with the first 10 rows!


In [19]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

def summarize_text(text, max_length=80):  # Increased max_length
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding="longest")
    
    summary_ids = model.generate(
        inputs['input_ids'], 
        max_length=max_length, 
        min_length=10,  # Increased min_length
        length_penalty=1.0,  # Lowered length penalty
        num_beams=4, 
        early_stopping=False  # Let it generate until it feels complete
    )
    
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


# Apply summarization to 'uses' column and store in a new 'summary' column
First10['summary'] = First10['uses'].dropna().apply(lambda x: summarize_text(x))

#idha n7ebou ne5dmou dataset lkol, nbadlou First10 par final_df

# Save the results to CSV
# Keep only 'drug_name' and 'summary' clumns
summary = First10[['drug_name', 'summary']]

# Save the final dataset
summary.to_csv('summary.csv', index=False)


C:\Users\agood\AppData\Local\Temp\ipykernel_30996\1146331443.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  First10['summary'] = First10['uses'].dropna().apply(lambda x: summarize_text(x))


In [ ]:
#from transformers import BartTokenizer, BartForConditionalGeneration

#model_name = "facebook/bart-large-cnn"
#tokenizer = BartTokenizer.from_pretrained(model_name)
#model = BartForConditionalGeneration.from_pretrained(model_name)

#def summarize_text(text, max_length=40):
    # Tokenize input text with truncation
   # inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding="longest")
    
    # Generate summary ids
   # summary_ids = model.generate(inputs['input_ids'], max_length=max_length, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    
    # Decode the summary
    #summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    #return summary

# Apply summarization to 'uses' column and store in a new 'summary' column
#cleaned['summary'] = cleaned['uses'].dropna().apply(lambda x: summarize_text(x))

# Save the results to CSV
#cleaned.to_csv('medications_with_summaries.csv', index=False)

#print("Summarization complete! Check 'medications_with_summaries1.csv' for the results.")
